# 1. SLR

In [2]:
import copy
import time

In [3]:
class LRParser:
    def __init__(self, grammar, terminals, non_terminals, start, dot):
        self.grammar = grammar
        self.terminals = terminals
        self.non_terminals = non_terminals
        self.start = start
        self.dot = dot
        
        self.first_table = {}
        self.follow_table = {}
        self.in_progress = set()     # to avoid left recursive when calculating first
        self.calculateFirstTable()
        self.calculateFollowTable()
        
        self.augmented_rules = []    # format of rule: [rhs, [<lhs symbol>]
        self.state_map = {}          # store rules of a state (format: state_count:[[rule1], [rule2], ...])
        self.state_dict = {}         # store which state go to which state
        self.state_count = 0
        self.initialAugmentation()
        self.generateStates()

        self.parse_table = []
        self.createParseTable()

    
    def shift(self, current_node, next_state, symbol):
        pass


    def reduce(self, current_node, rule):
        pass

    
    def initialAugmentation(self):
        for key in grammar.keys():
            lhs, rhs = grammar[key]
            new_rhs = [self.dot]
            for elem in rhs:
                new_rhs.append(elem)
            self.augmented_rules.append([lhs, new_rhs])

    def generateStates(self):
        
        # generate the first state I_0
        first_state = []
        for rule in self.augmented_rules:
            if rule[0] == self.start:
                first_state.append(rule)
        closure_rules = self.findClosure(first_state)
        self.state_dict[0] = closure_rules

        # generate states until no more state is able to be generated
        prev_len = -1
        state_completed_GOTO = []
        while prev_len != len(self.state_dict):
            prev_len = len(self.state_dict)

            keys = list(self.state_dict.keys())
            for state in keys:
                if state not in state_completed_GOTO:
                    self.computeGOTO(state)
                    state_completed_GOTO.append(state)

    
    def computeGOTO(self, state):
        generate_new_state_for = []
        for rule in self.state_dict[state]:
            # if the rule ends with dot (can't shift anymore) => skip
            if rule[1][-1] == self.dot:
                continue

            dot_ind = rule[1].index(self.dot)
            next_sym = rule[1][dot_ind+1]

            if next_sym not in generate_new_state_for:
                generate_new_state_for.append(next_sym)

        for sym in generate_new_state_for:
            self.GOTO(state, sym)

    
    def GOTO(self, state, sym):
        new_state = []
        for rule in self.state_dict[state]:
            # if the rule ends with dot (can't shift anymore) => skip
            if rule[1][-1] == self.dot:
                continue

            dot_ind = rule[1].index(self.dot)
            next_sym = rule[1][dot_ind+1]

            # shift operation from the previous state of rule on that
            if next_sym == sym:
                # swap dot with next_sym
                shifted_rule = copy.deepcopy(rule)
                shifted_rule[1][dot_ind] = shifted_rule[1][dot_ind + 1]
                shifted_rule[1][dot_ind + 1] = self.dot
                new_state.append(shifted_rule)

        closure_rules = self.findClosure(new_state)

        # check if state exist
        state_exists = -1
        for state_num in self.state_dict:
            if self.state_dict[state_num] == new_state:
                state_exists = state_num
                break
     
        # stateMap is a mapping of GOTO with
        # its output states
        if state_exists == -1:
            self.state_count += 1
            self.state_dict[self.state_count] = closure_rules
            self.state_map[(state, sym)] = self.state_count
        else:
            self.state_map[(state, sym)] = state_exists
            

    def findClosure(self, closure_rules):
        # generate closure for the rules in new_state
        # generate until can't generate anymore
        prev_len = -1
        while prev_len != len(closure_rules):
            prev_len = len(closure_rules)
            for rule in closure_rules:
                if rule[1][-1] == self.dot:
                    continue
                    
                dot_ind = rule[1].index(self.dot)
                next_sym = rule[1][dot_ind+1]
    
                # if next_sym is non_terminal then continue adding rule with that nonterminals as lhs
                if next_sym in self.non_terminals:
                    for augmented_rule in self.augmented_rules:
                        if augmented_rule[0] == next_sym and augmented_rule not in closure_rules:
                            closure_rules.append(augmented_rule)
        return closure_rules

        
    def calculateFirstTable(self):
        for key in grammar.keys():
            rule = grammar[key]
            lhs, rhs = rule

            if lhs not in self.first_table:
                self.first_table[lhs] = list(elem for elem in self.first(rule))
            else:
                res = self.first(rule)
                for elem in res:
                    if elem not in self.first_table[lhs]:
                        self.first_table[lhs].append(elem)

    
    def calculateFollowTable(self):
        for nt in self.non_terminals:
            self.follow_table[nt] = self.follow(nt)

    
    def first(self, rule):
        lhs, rhs = rule
        
        if lhs in self.in_progress:
            return []  # prevent infinite recursion
        
        # mark this non-terminal as being processed
        self.in_progress.add(lhs)
        
        # rule for terminals
        if rhs[0] in terminals:
            return [rhs[0]]
            
        # rule for epsilon
        elif rhs[0] == "#":
            return ["#"]
            
        # rule for non-terminal
        else:
            res = []
            for key in grammar.keys():
                if rhs[0] == grammar[key][0]:
                    for elem in self.first(grammar[key]):
                        res.append(elem) 

            if "#" in res:
                res.remove("#")
                
            self.in_progress.remove(lhs)  # finished processing this non-terminal
            return res

    
    def follow(self, nt):
        res = set()

        # for start symbol return $
        if nt == self.start:
            res.add("$")

        for key in grammar.keys():
            lhs, rhs = grammar[key]
            
            for i, symbol in enumerate(rhs):
                if symbol == nt:
                    rhs = rhs[i + 1:]

                    # rule 2: there is a symbol after nt
                    if len(rhs) != 0:
                        # if the symbol after nt is also a non-terminal:
                        #   - calculate its first (remove epsilon) and add to res
                        #   - if its first contain epsilon, then continue checking the next symbol
                        # else the symbol after nt is a terminal:
                        #   - then add it to res
                        for sym in rhs:
                            if sym in self.terminals:
                                res.add(sym)
                                break
                            elif sym in self.first_table:
                                first_sym = self.first_table[sym]
                                res.update(set(first_sym) - {"#"})
    
                                if "#" in first_sym:
                                    res.remove("#")
                                else:
                                    break

                    # rule 3: there is no symbol after nt -> FOLLOW(lhs) ⊆ FOLLOW(nt)
                    if len(rhs) == 0:  
                        if lhs != nt:
                            res.update(self.follow(lhs))
                                
        return list(res)

    def createParseTable(self):
        rows = list(self.state_dict.keys())
        cols = self.terminals + ["$"] + self.non_terminals

        # create empty table
        temp_row = []
        for i in range(len(cols)):
            temp_row.append([])
        for i in range(len(rows)):
            self.parse_table.append(copy.deepcopy(temp_row))

        # add shift and goto entries to table
        for entry in self.state_map.keys():
            state = entry[0]
            sym = entry[1]

            row_ind = rows.index(state)
            col_ind = cols.index(sym)

            if sym in self.terminals:
                self.parse_table[row_ind][col_ind].append(f"S{self.state_map[entry]}")
            elif sym in self.non_terminals:
                self.parse_table[row_ind][col_ind].append(f"G{self.state_map[entry]}")

        # add reduce to table
        for state in self.state_dict.keys():
            for rule in self.state_dict[state]:
                # if the rule is a handle -> add reduce correspondingly
                if rule[1][-1] == self.dot:
                    copy_rhs = copy.deepcopy(rule[1])
                    copy_rhs.remove(self.dot)

                    # add entry R_rule_num (Reduce -> rule_num) to entry (state, follow(rhs)) in parse table
                    for rule_num in self.grammar.keys():
                        if grammar[rule_num][0] == rule[0] and grammar[rule_num][1] == copy_rhs:
                            for follow in self.follow_table[rule[0]]:
                                row_ind = rows.index(state)
                                col_ind = cols.index(follow)
                                if rule_num == 0:
                                    self.parse_table[row_ind][col_ind].append("Accept")
                                else:
                                    self.parse_table[row_ind][col_ind].append(f"R{rule_num}")

    	# printing table
        print("\nParsing table:\n")
        frmt = "{:>8}" * len(cols)
        print(" ", frmt.format(*cols), "\n")
        ptr = 0
        j = 0
        for y in self.parse_table:
            # frmt1 = "{:>8}"
            print(f"{{:>3}}".format('I'+str(j)), end="")
            for e in y:
                print(f"{{:>8}}".format("/".join(e)), end="")
            print()
            j += 1

    def printResultAndGoto(self):
        print("\nStates Generated: \n")
        for st in self.state_dict:
            print(f"State = I{st}")
            self.printResult(self.state_dict[st])
            print()# print goto states
        print("\nStates Generated: \n")
        for st in self.state_dict:
            print(f"State = I{st}")
            self.printResult(self.state_dict[st])
            print()

        print("Result of GOTO computation:\n")
        self.printAllGOTO(self.state_map)

    

    def printResult(self, rules):
        for rule in rules:
            print(f"{rule[0]} ->"
                  f" {' '.join(rule[1])}")

    def printAllGOTO(self, diction):
        for itr in diction:
            print(f"GOTO ( I{itr[0]} ,"
                  f" {itr[1]} ) = I{self.state_map[itr]}")


In [4]:
class SLRParser(LRParser):
    def __init__(self, grammar, terminals, non_terminals, start, dot):
        super().__init__(grammar, terminals, non_terminals, start, dot)

    def parse(self, input_string):
        # self.printResultAndGoto()
        print(f"\n\nParsing input string: {input_string}\n")
        rows = list(self.state_dict.keys())
        cols = self.terminals + ["$"] + self.non_terminals
        
        ls_input = input_string + ["$"]
        current_char = ls_input[0]
        ls_output = []
        stack = [0]
        while True:
            # print(ls_input, current_char, stack)
            # time.sleep(1)
            
            row_ind = rows.index(stack[-1])
            col_ind = cols.index(current_char)
            
            operation = self.parse_table[row_ind][col_ind][0]  # just get the first operation in conflict
            
            if operation == []:
                print("Input not accepted")
                return 
                
            else:
                # print(operation)
                # reduce operation
                if operation[0] == "R":
                    rule_num = int(operation[1:])
                    current_char = self.grammar[rule_num][0]
                    
                    # pop stack equal to number of char on rhs of reduce rule
                    stack_pop_count = len(self.grammar[rule_num][1])
                    stack = stack[:-stack_pop_count]

                    ls_output.append(rule_num)
                
                # goto operation
                elif operation[0] == "G":
                    stack.append(int(operation[1:]))
                    current_char = ls_input[0]  
                    
                # shift operation
                elif operation[0] == "S":
                    stack.append(int(operation[1:]))
                    ls_input.pop(0) 
                    current_char = ls_input[0]      

                # accept reached
                elif operation == "Accept":
                    print("\nInput accepted")
                    return 

    
# Example 1 Grammar and Tables
grammar = {
    0: ("E'", ["E"]),                # Rule 0: E'→ E
    1: ("E", ["E", "+", "T"]),       # Rule 1: E → E + T
    2: ("E", ["T"]),                 # Rule 2: E → T
    3: ("T", ["T", "*", "F"]),       # Rule 3: T → T * F
    4: ("T", ["F"]),                 # Rule 4: T → F
    5: ("F", ["(", "E", ")"]),       # Rule 5: F → ( E )
    6: ("F", ["a"]),                 # Rule 6: F → a
    
}

terminals = ["a", "+", "*","(", ")"]
non_terminals = ["E'", "E", "T", "F"]
start = "E'"
dot = '·'

# Test the Parser
parser = SLRParser(grammar, terminals, non_terminals, start, dot)
input_string = list("a*a+a*a+a")
parser.parse(input_string)


# # Example 2 Grammar and Tables
# grammar = {
#     1: ("S", ["L", "=", "R"]),    # Rule 1: S → L = R
#     2: ("S", ["R"]),              # Rule 2: S → R
#     3: ("L", ["*", "R"]),         # Rule 3: L → * R
#     4: ("L", ["a"]),              # Rule 4: L → a
#     5: ("R", ["L"]),              # Rule 5: R → L
# }

# terminals = ["a", "=", "*"]
# non_terminals = ["S", "L", "R"]
# start = "S"
# dot = '·'

# # Test the Parser
# parser = SLRParser(grammar, terminals, non_terminals, start, dot)
# input_string = list("a=a")
# parser.parse(input_string)





Parsing table:

         a       +       *       (       )       $      E'       E       T       F 

 I0      S5                      S4                              G1      G2      G3
 I1              S6                          Accept                                
 I2              R2      S7              R2      R2                                
 I3              R4      R4              R4      R4                                
 I4      S5                      S4                              G8      G2      G3
 I5              R6      R6              R6      R6                                
 I6      S5                      S4                                      G9      G3
 I7      S5                      S4                                             G10
 I8              S6                     S11                                        
 I9              R1      S7              R1      R1                                
I10              R3      R3              R3      R3       

In [5]:
# Example 3 Grammar and Tables
grammar = {
    0: ("S'", ["S"]),                # Rule 0: S'→ S
    1: ("S", ["a", "b", "C"]),       # Rule 1: S → abC
    2: ("S", ["a", "B", "C"]),       # Rule 2: S → aBC
    3: ("B", ["b"]),                 # Rule 3: B → b
    4: ("C", ["d"]),                 # Rule 4: C → d
    
}

terminals = ["a", "b", "d"]
non_terminals = ["S'", "S", "B", "C"]
start = "S'"
dot = '·'

# Test the Parser
parser = SLRParser(grammar, terminals, non_terminals, start, dot)
# input_string = list("a*a+a*a+a")
# parser.parse(input_string)


Parsing table:

         a       b       d       $      S'       S       B       C 

 I0      S2                                      G1                
 I1                          Accept                                
 I2              S3                                      G4        
 I3                   S6/R3                                      G5
 I4                      S6                                      G7
 I5                              R1                                
 I6                              R4                                
 I7                              R2                                


# 2. GLR

In [8]:
class GSSNode:
    def __init__(self, state, symbol):
        self.state = state  # Parser state
        self.symbol = symbol  # Grammar symbol
        self.predecessor_edges = []  # List of edges to predecessor nodes
        self.successor_edges = []  # List of edges to successor  nodes

    def add_predecessor(self, predecessor):
        """Add an edge to a predecessor node."""
        self.predecessor_edges.append(predecessor)

    def add_successor(self, successor ):
        """Add an edge to a predecessor node."""
        self.successor_edges.append(successor)


class GSS:
    def __init__(self):
        self.current_v = 0
        self.nodes = []  # Map v_number -> GSSNode

    def create_node(self, state, symbol):
        self.current_v += 1
        self.nodes[self.current_v] = GSSNode(state, symbol)
        return self.nodes[self.current_v]

    def add_edge(self, node, successor):
        # successor of v = there is an edge from v to this node 
        # opposite with edge direction (which is <- in book diagram)
        node.add_successor(successor)
        successor.add_predecessor(node)

    # def visualize(self):
    #     """Optional: Visualize the GSS for debugging."""
    #     for key, node in self.nodes.items():
    #         edges = ", ".join([f"({e.state}, {e.symbol})" for e in node.edges])
    #         print(f"Node ({node.state}, {node.symbol}): Edges -> {edges}")


class GLRParser(LRParser):
    def __init__(self, grammar, terminals, non_terminals, start, dot):
        super().__init__(grammar, terminals, non_terminals, start, dot)
        self.gss = GSS()
        self.root_node = self.gss.get_or_create_node(0, "")  # Initial state
        self.input_string = []
        self.r = False
        self.U = {0: [self.root_node]}
        self.R = []
        self.Q = []
        self.A = []
        

    # def shift(self, current_node, next_state, symbol):
    #     """Perform the shift operation."""
    #     new_node = self.gss.get_or_create_node(next_state, symbol)
    #     self.gss.add_edge(new_node, current_node)
    #     return new_node

    # def reduce(self, current_node, rule):
    #     """Perform the reduce operation."""
    #     lhs, rhs = rule
    #     for _ in rhs:  # Pop the right-hand side
    #         if current_node.edges:
    #             current_node = current_node.edges[0]  # Follow one predecessor
    #         else:
    #             break

    #     # Create a new node for the reduced non-terminal
    #     reduced_node = self.gss.get_or_create_node(current_node.state, lhs)
    #     self.gss.add_edge(reduced_node, current_node)
    #     return reduced_node

    def parse(self, input_string):
        """Main parsing loop."""
        self.input_string.append("$")

        i = 0
        while i < len(self.input_string):
            self.parseword(i, input_string)
            i += 1

        if self.r == True:
            print("Input accepted!")
        else:
            print("Input not accepted!")
        return self.r

    def parseword(self, i, input_string):
        self.A.append(self.U[i])
        self.R = []
        self.Q = []

        while len(self.R) != 0 or len(self.A) != 0:
            if len(self.A) != 0:
                self.actor(i, input_string)
            elif len(self.R) != 0:
                self.reducer(i, input_string)


        self.shifter(i)

    def actor(self, i, input_string):
        v = self.A[0].pop(0)
            
        current_state = v.state
        symbol = input_string[i]

        if (current_state, symbol) in self.parse_table:
            operation = self.parse_table[(current_state, symbol)]
            if len(operation) == 0:
                return
            
            if operation[0] == "Accept":
                self.r = True
            elif operation[0] == "S":
                self.Q.append(v, operation[1:])
            elif operation[0] == "R":
                # elif operation = "reduce p"
                # for all x such that x SUCCESSORS(v), add [v, x, p] to R.
                for key in self.gss.nodes.keys():
                    if current_state == self.gss.nodes[key].state and symbol == self.gss.nodes[key].symbol:
                        x = self.gss.nodes[key]
                        for v in x.successor_edges:
                            self.R.append([v, x, action[1:]])

    
    def reducer(self, i, input_string):
        (v, x, p) = self.R.pop(0)
        N = self.grammar[int(p)][0]

        # for all w such that there exists a path of length 2|p|-1 from x to w do
        for ___:
            operation = self.parse_table(w.state, N)
             
            if len(operation) == 0:
                return
            elif operation[0] == "G":
                s = operation[1:]
             

            # if there exists u such that u in U_i and STATE(u) = s then
            need_reduce = false
            for u in self.U[i]:
                if u.state == s:
                    need_reduce = True
                    break
                    
            # if there exists u such that u in U_i and STATE(u) = s then
            if need_reduce:
                # if there already exists a path of length 2 from u to w then
                if ___:
                    continue
                else:
                    # if u in A
                    # for all q such that reduce q in  ACTION(STATE(u),ai+1)
                    # add (u,z,q) to R.
                    if u in A:
                        pass
            else:
                pass
                        
                    
    

    def shifter(self, i, input_string):
        all_s = list(set([s for v, s in Q]))
        x = input_string[i+1]
        
        for state in all_s:
            # new state node after shift
            w = self.gss.create_node(int(state), x)

            # add w to U_i+1
            if i+1 in self.U.keys():
                self.U[i+1] = []
            self.U[i+1].append(w)

            # add edge between w and v in <v,s> with the s above
            all_v = list(set([v for v, s in Q if s == state]))
            for v in all_v:
                self.gss.add_edge(v, w)
        pass

# Example 1 Grammar and Tables
grammar = {
    0: ("S'", ["S"]),                # Rule 0: S'→ S
    1: ("S", ["a", "b", "C"]),       # Rule 1: S → abC
    2: ("S", ["a", "B", "C"]),       # Rule 2: S → aBC
    3: ("B", ["b"]),                 # Rule 3: B → b
    4: ("C", ["d"]),                 # Rule 4: C → d
    
}

terminals = ["a", "b", "d"]
non_terminals = ["S'", "S", "B", "C"]
start = "S'"
dot = '·'

# Test the Parser
parser = SLRParser(grammar, terminals, non_terminals, start, dot)
# input_string = list("abd")
# parser.parse(input_string)


<class 'SyntaxError'>: cannot assign to attribute here. Maybe you meant '==' instead of '='? (<ipython-input-8-9e4b6ed1b5ef>, line 141)